In [1]:
import pandas as pd
import csv
import os
import sys
import csv
import time
import numpy as np
import random
from scipy.io import arff

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

sys.path.insert(0, f"{os.path.dirname(os.getcwd())}/src")
from utils import evaluate_result
from data_imbalance_src.smote_oversampling import RandomOversampling, ADASYNOversampling, BorderlineSMOTEOversampling, SMOTEOversampling, SVMSMOTEOversampling
from data_imbalance_src.smote_oversampling import SMOTUNEDOversampling
from data_imbalance_src.dazzle import DAZZLEOversampling
from data_imbalance_src.Imbalance_Farou2022.data_generation import GANOversampling
from data_imbalance_src.random_projection import RandomProjectionOversampling

2023-09-19 15:41:29.851213: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-19 15:41:29.860592: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-19 15:41:29.990869: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-19 15:41:29.993685: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-19 15:41:30.787259: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

# JavaScript_Vulnerability

In [22]:
rs = random.randint(0, 100000)

In [ ]:
write_file = f"JS_Vuln_res_r1_rn{rs}.csv"
write_path = f"{os.path.dirname(os.getcwd())}/result/{write_file}"
with open(write_path, "w", newline="") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["oversampling_scheme", "runtime", "learner", "acc", "prec", "recall", "fpr", "f1", "auc", "g_score", "d2h"])

In [23]:
data_path = f"{os.path.dirname(os.getcwd())}/data/JavaScript_Vulnerability/"
datafiles = [f for f in os.listdir(data_path) if f.endswith("csv")]
datafiles

['JSVulnerabilityDataSet-1.0.csv']

In [24]:
df = pd.read_csv(f"{data_path}/{datafiles[0]}")
drop_columns = ["name", "longname", "path", "full_repo_path", "line", "column", "endline", "endcolumn"]
df = df.drop(drop_columns, axis=1)
df = df.drop_duplicates()
df.reset_index(inplace=True, drop=True)

In [25]:
df

,CC,CCL,CCO,CI,CLC,CLLC,McCC,NL,NLE,CD,...,HVOC,HDIFF,HVOL,HEFF,HBUGS,HTIME,CYCL,PARAMS,CYCL_DENS,Vuln
0,0.0,0,0,0,0.0,0,3,1,1,0.000000,...,21,10.250000,342.600759,3511.657780,0.114200,195.092099,3,2,30.000000,1
1,0.0,0,0,0,0.0,0,11,3,3,0.041667,...,48,21.500000,893.594000,19212.271002,0.297865,1067.348389,9,3,37.500000,1
2,0.0,0,0,0,0.0,0,1,0,0,0.000000,...,10,1.928571,43.185065,83.285483,0.014395,4.626971,1,4,100.000000,1
3,0.0,0,0,0,0.0,0,1,0,0,0.000000,...,10,1.928571,43.185065,83.285483,0.014395,4.626971,1,4,100.000000,0
4,0.0,0,0,0,0.0,0,3,2,2,0.000000,...,37,8.703704,416.756269,3627.323084,0.138919,201.517949,3,3,30.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6266,0.0,0,0,0,0.0,0,2,1,1,0.000000,...,20,3.857143,142.623627,550.119705,0.047541,30.562206,2,2,33.333333,0
6267,0.0,0,0,0,0.0,0,2,1,1,0.000000,...,12,3.250000,75.284213,244.673691,0.025095,13.592983,2,2,40.000000,0
6268,0.0,0,0,0,0.0,0,6,4,4,0.000000,...,49,13.800000,724.297570,9995.306464,0.241433,555.294804,6,1,31.578947,1
6269,0.0,0,0,0,0.0,0,2,1,1,0.000000,...,19,6.071429,267.619433,1624.832274,0.089206,90.268460,2,2,18.181818,0


In [26]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
print("y value counts: \n", str(y.value_counts()))
print("y class ratio: 1:", str(round(y.value_counts()[0]/y.value_counts()[1])))

y value counts: 
 0    5367
1     904
Name: Vuln, dtype: int64
y class ratio: 1: 6


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=rs)
print("--- y train classes count: \n" + str(y_train.value_counts()))
print("--- y train ratio: 1:" + str(round(y_train.value_counts()[0] / y_train.value_counts()[1])))
print(" ")
print("--- y test classes count: \n" + str(y_test.value_counts()))
print("--- y test ratio: 1:" + str(round(y_test.value_counts()[0] / y_test.value_counts()[1])))

--- y train classes count: 
0    4293
1     723
Name: Vuln, dtype: int64
--- y train ratio: 1:6
 
--- y test classes count: 
0    1074
1     181
Name: Vuln, dtype: int64
--- y test ratio: 1:6


# Moodle_Vulnerability

In [2]:
write_file = "Moodle_Vuln_res_r10.csv"
write_path = f"{os.path.dirname(os.getcwd())}/result/{write_file}"
with open(write_path, "w", newline="") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["oversampling_scheme", "runtime", "learner", "acc", "prec", "recall", "fpr", "f1", "auc", "g_score", "d2h"])

In [3]:
data_path = f"{os.path.dirname(os.getcwd())}/data/Vulnerable_Files/moodle-2_0_0-metrics.arff"
data = arff.loadarff(data_path)
df = pd.DataFrame(data[0])
df['IsVulnerable'] = df['IsVulnerable'].astype('str')
d = {'b\'yes\'': 1, 'b\'no\'': 0}
df['IsVulnerable'] = df['IsVulnerable'].astype(str).map(d).fillna(df['IsVulnerable'])
df = df.drop_duplicates()
df.reset_index(inplace=True, drop=True)

In [4]:
df

,nonecholoc,loc,nmethods,ccomdeep,ccom,nest,hvol,nIncomingCalls,nIncomingCallsUniq,nOutgoingInternCalls,nOutgoingExternFlsCalled,nOutgoingExternFlsCalledUniq,nOutgoingExternCallsUniq,IsVulnerable
0,58.0,58.0,0.0,12.0,12.0,2.0,490.191513,0.0,0.0,0.0,36.0,29.0,23.0,0
1,90.0,107.0,1.0,20.0,20.0,5.0,1473.469064,0.0,0.0,0.0,106.0,69.0,50.0,0
2,38.0,43.0,0.0,8.0,8.0,2.0,419.666089,0.0,0.0,0.0,69.0,54.0,26.0,0
3,130.0,130.0,0.0,26.0,26.0,4.0,2262.272730,0.0,0.0,0.0,100.0,70.0,84.0,0
4,96.0,96.0,1.0,13.0,13.0,5.0,1359.742007,0.0,0.0,1.0,94.0,66.0,42.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2051,20.0,20.0,3.0,1.0,1.0,0.0,113.676138,569.0,567.0,1.0,1049.0,1020.0,3.0,0
2052,61.0,61.0,5.0,8.0,8.0,2.0,539.859175,561.0,556.0,3.0,1091.0,1029.0,28.0,0
2053,43.0,43.0,0.0,6.0,6.0,1.0,460.843316,0.0,0.0,0.0,56.0,39.0,30.0,0
2054,19.0,19.0,3.0,1.0,1.0,0.0,95.006064,568.0,567.0,2.0,1022.0,1018.0,2.0,0


In [5]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
print("y value counts: \n", str(y.value_counts()))
print("y class ratio: 1:", str(round(y.value_counts()[0]/y.value_counts()[1])))

y value counts: 
 0    2032
1      24
Name: IsVulnerable, dtype: int64
y class ratio: 1: 85


In [6]:
rs = random.randint(0, 100000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=rs)
print("--- y train classes count: \n" + str(y_train.value_counts()))
print("--- y train ratio: 1:" + str(round(y_train.value_counts()[0] / y_train.value_counts()[1])))
print(" ")
print("--- y test classes count: \n" + str(y_test.value_counts()))
print("--- y test ratio: 1:" + str(round(y_test.value_counts()[0] / y_test.value_counts()[1])))

--- y train classes count: 
0    1625
1      19
Name: IsVulnerable, dtype: int64
--- y train ratio: 1:86
 
--- y test classes count: 
0    407
1      5
Name: IsVulnerable, dtype: int64
--- y test ratio: 1:81


# Defect Prediction

In [2]:
write_file = "defect_prediction_r3.csv"
write_path = f"{os.path.dirname(os.getcwd())}/result/{write_file}"
with open(write_path, "w", newline="") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["oversampling_scheme", "runtime", "learner", "acc", "prec", "recall", "fpr", "f1", "auc", "g_score", "d2h"])

In [3]:
data_path = f"{os.path.dirname(os.getcwd())}/data/imbalance_defects_prediction/7_CK_NET_PROC/input/Eclipse_JDT_Core--CK_NET_PROC.arff"
data = arff.loadarff(data_path)
df = pd.DataFrame(data[0])
df['isBug'] = df['isBug'].astype('str')
d = {'b\'YES\'': 1, 'b\'NO\'': 0}
df['isBug'] = df['isBug'].astype(str).map(d).fillna(df['isBug'])
df = df.drop_duplicates()
df.reset_index(inplace=True, drop=True)

In [4]:
df

,wmc,dit,rfc,noc,cbo,lcom,loc,revision_num,author_num,linesadd_sum,...,InFreeClo,OutValClo,InValClo,OutRecipClo,InRecipClo,OutdwReach,IndwReach,nOutdwReach,nIndwReach,isBug
0,18.0,1.0,46.0,0.0,7.0,78.0,82.0,26.0,4.0,332.0,...,0.002014,0.953028,0.502399,0.206168,0.134612,206.300583,134.578400,0.206921,0.134983,0
1,5.0,1.0,5.0,0.0,2.0,10.0,18.0,22.0,4.0,192.0,...,0.002019,0.000000,0.503744,0.001003,0.153207,1.000000,153.099930,0.001003,0.153561,0
2,14.0,1.0,28.0,0.0,7.0,6.0,50.0,16.0,6.0,69.0,...,0.001003,0.955744,0.000000,0.242747,0.001003,242.734924,1.000000,0.243465,0.001003,0
3,13.0,1.0,21.0,1.0,4.0,45.0,69.0,43.0,8.0,485.0,...,0.002016,0.952940,0.503016,0.199539,0.182509,199.697235,182.283066,0.200298,0.182832,0
4,1.0,1.0,2.0,0.0,1.0,1.0,3.0,14.0,4.0,55.0,...,0.002020,0.000000,0.503914,0.001003,0.168100,1.000000,167.933151,0.001003,0.168438,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,84.0,2.0,126.0,1.0,6.0,66.0,276.0,5.0,1.0,23.0,...,0.002011,0.952831,0.501852,0.205453,0.100706,205.588043,100.808090,0.206207,0.101111,0
993,50.0,2.0,51.0,1.0,6.0,45.0,121.0,7.0,1.0,9.0,...,0.002013,0.953497,0.502353,0.248260,0.124990,248.223679,124.995102,0.248971,0.125371,0
994,12.0,2.0,25.0,0.0,4.0,36.0,48.0,3.0,1.0,2.0,...,0.002011,0.952784,0.501867,0.200952,0.101158,201.104736,101.258087,0.201710,0.101563,0
995,21.0,1.0,29.0,2.0,5.0,55.0,81.0,7.0,1.0,12.0,...,0.002022,0.258222,0.504385,0.065971,0.086352,65.966690,86.514275,0.066165,0.086775,0


In [5]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
print("y value counts: \n", str(y.value_counts()))
print("y class ratio: 1:", str(round(y.value_counts()[0]/y.value_counts()[1])))

y value counts: 
 0    791
1    206
Name: isBug, dtype: int64
y class ratio: 1: 4


In [6]:
rs = random.randint(0, 100000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=rs)
print("--- y train classes count: \n" + str(y_train.value_counts()))
print("--- y train ratio: 1:" + str(round(y_train.value_counts()[0] / y_train.value_counts()[1])))
print(" ")
print("--- y test classes count: \n" + str(y_test.value_counts()))
print("--- y test ratio: 1:" + str(round(y_test.value_counts()[0] / y_test.value_counts()[1])))

--- y train classes count: 
0    632
1    165
Name: isBug, dtype: int64
--- y train ratio: 1:4
 
--- y test classes count: 
0    159
1     41
Name: isBug, dtype: int64
--- y test ratio: 1:4


### Normal Run

In [15]:
# normal run - without any oversampling technique
# inputs: X_train, y_train, X_test, y_test

# scale data
scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

# create models
clf_SVM = SVC()
clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
clf_DT = DecisionTreeClassifier()
clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

clf_SVM.fit(X_train_scale, y_train)
clf_KNN.fit(X_train_scale, y_train)
clf_LR.fit(X_train_scale, y_train)
clf_DT.fit(X_train_scale, y_train)
clf_RF.fit(X_train_scale, y_train)
clf_LightGBM.fit(X_train_scale, y_train)
clf_Adaboost.fit(X_train_scale, y_train)
clf_GBDT.fit(X_train_scale, y_train)

y_pred_SVM = clf_SVM.predict(X_test_scale)
y_pred_KNN = clf_KNN.predict(X_test_scale)
y_pred_LR = clf_LR.predict(X_test_scale)
y_pred_DT = clf_DT.predict(X_test_scale)
y_pred_RF = clf_RF.predict(X_test_scale)
y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
y_pred_GBDT = clf_GBDT.predict(X_test_scale)

In [8]:
with open(f"{os.path.dirname(os.getcwd())}/result/{write_file}", "a", newline="") as f:
    csv_writer = csv.writer(f)
    
    csv_writer.writerow(["No", 0, "SVM"] + evaluate_result(y_pred_SVM, y_test))
    csv_writer.writerow(["No", 0, "KNN"] + evaluate_result(y_pred_KNN, y_test))
    csv_writer.writerow(["No", 0, "LR"] + evaluate_result(y_pred_LR, y_test))
    csv_writer.writerow(["No", 0, "DT"] + evaluate_result(y_pred_DT, y_test))
    csv_writer.writerow(["No", 0, "RF"] + evaluate_result(y_pred_RF, y_test))
    csv_writer.writerow(["No", 0, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
    csv_writer.writerow(["No", 0, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
    csv_writer.writerow(["No", 0, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

### Random Oversampling

In [9]:
# random oversampling run - random oversampling technique
# inputs: X_train_random, y_train_random, X_test, y_test

rt, X_train_new, y_train_new = RandomOversampling(X_train=X_train, y_train=y_train)

# scale data
scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

# create models
clf_SVM = SVC()
clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
clf_DT = DecisionTreeClassifier()
clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

clf_SVM.fit(X_train_new, y_train_new)
clf_KNN.fit(X_train_new, y_train_new)
clf_LR.fit(X_train_new, y_train_new)
clf_DT.fit(X_train_new, y_train_new)
clf_RF.fit(X_train_new, y_train_new)
clf_LightGBM.fit(X_train_new, y_train_new)
clf_Adaboost.fit(X_train_new, y_train_new)
clf_GBDT.fit(X_train_new, y_train_new)

y_pred_SVM = clf_SVM.predict(X_test)
y_pred_KNN = clf_KNN.predict(X_test)
y_pred_LR = clf_LR.predict(X_test)
y_pred_DT = clf_DT.predict(X_test)
y_pred_RF = clf_RF.predict(X_test)
y_pred_LightGBM = clf_LightGBM.predict(X_test)
y_pred_Adaboost = clf_Adaboost.predict(X_test)
y_pred_GBDT = clf_GBDT.predict(X_test)

y train ratio: 1:1


In [10]:
with open(f"{os.path.dirname(os.getcwd())}/result/{write_file}", "a", newline="") as f:
    csv_writer = csv.writer(f)
    
    csv_writer.writerow(["Random", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
    csv_writer.writerow(["Random", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
    csv_writer.writerow(["Random", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
    csv_writer.writerow(["Random", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
    csv_writer.writerow(["Random", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
    csv_writer.writerow(["Random", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
    csv_writer.writerow(["Random", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
    csv_writer.writerow(["Random", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

### ADASYN Oversampling

In [11]:
# ADASYN oversampling run - ADASYN oversampling technique
# inputs: X_train_random, y_train_random, X_test, y_test

rt, X_train_new, y_train_new = ADASYNOversampling(X_train=X_train, y_train=y_train)

# scale data
scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

# create models
clf_SVM = SVC()
clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
clf_DT = DecisionTreeClassifier()
clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

clf_SVM.fit(X_train_new, y_train_new)
clf_KNN.fit(X_train_new, y_train_new)
clf_LR.fit(X_train_new, y_train_new)
clf_DT.fit(X_train_new, y_train_new)
clf_RF.fit(X_train_new, y_train_new)
clf_LightGBM.fit(X_train_new, y_train_new)
clf_Adaboost.fit(X_train_new, y_train_new)
clf_GBDT.fit(X_train_new, y_train_new)

y_pred_SVM = clf_SVM.predict(X_test)
y_pred_KNN = clf_KNN.predict(X_test)
y_pred_LR = clf_LR.predict(X_test)
y_pred_DT = clf_DT.predict(X_test)
y_pred_RF = clf_RF.predict(X_test)
y_pred_LightGBM = clf_LightGBM.predict(X_test)
y_pred_Adaboost = clf_Adaboost.predict(X_test)
y_pred_GBDT = clf_GBDT.predict(X_test)

y train ratio: 1:1


In [12]:
with open(f"{os.path.dirname(os.getcwd())}/result/{write_file}", "a", newline="") as f:
    csv_writer = csv.writer(f)
    
    csv_writer.writerow(["ADASYN", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
    csv_writer.writerow(["ADASYN", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
    csv_writer.writerow(["ADASYN", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
    csv_writer.writerow(["ADASYN", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
    csv_writer.writerow(["ADASYN", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
    csv_writer.writerow(["ADASYN", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
    csv_writer.writerow(["ADASYN", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
    csv_writer.writerow(["ADASYN", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

### BorderlineSMOTE Oversampling

In [13]:
# BorderlineSMOTE oversampling run - BorderlineSMOTE oversampling technique
# inputs: X_train_random, y_train_random, X_test, y_test

rt, X_train_new, y_train_new = BorderlineSMOTEOversampling(X_train=X_train, y_train=y_train)

# scale data
scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

# create models
clf_SVM = SVC()
clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
clf_DT = DecisionTreeClassifier()
clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

clf_SVM.fit(X_train_new, y_train_new)
clf_KNN.fit(X_train_new, y_train_new)
clf_LR.fit(X_train_new, y_train_new)
clf_DT.fit(X_train_new, y_train_new)
clf_RF.fit(X_train_new, y_train_new)
clf_LightGBM.fit(X_train_new, y_train_new)
clf_Adaboost.fit(X_train_new, y_train_new)
clf_GBDT.fit(X_train_new, y_train_new)

y_pred_SVM = clf_SVM.predict(X_test)
y_pred_KNN = clf_KNN.predict(X_test)
y_pred_LR = clf_LR.predict(X_test)
y_pred_DT = clf_DT.predict(X_test)
y_pred_RF = clf_RF.predict(X_test)
y_pred_LightGBM = clf_LightGBM.predict(X_test)
y_pred_Adaboost = clf_Adaboost.predict(X_test)
y_pred_GBDT = clf_GBDT.predict(X_test)

y train ratio: 1:1


In [14]:
with open(f"{os.path.dirname(os.getcwd())}/result/{write_file}", "a", newline="") as f:
    csv_writer = csv.writer(f)
    
    csv_writer.writerow(["BorderlineSMOTE", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
    csv_writer.writerow(["BorderlineSMOTE", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
    csv_writer.writerow(["BorderlineSMOTE", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
    csv_writer.writerow(["BorderlineSMOTE", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
    csv_writer.writerow(["BorderlineSMOTE", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
    csv_writer.writerow(["BorderlineSMOTE", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
    csv_writer.writerow(["BorderlineSMOTE", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
    csv_writer.writerow(["BorderlineSMOTE", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

### SMOTE Oversampling

In [15]:
# SMOTE oversampling run - SMOTE oversampling technique
# inputs: X_train_random, y_train_random, X_test, y_test

rt, X_train_new, y_train_new = SMOTEOversampling(X_train=X_train, y_train=y_train)

# scale data
scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

# create models
clf_SVM = SVC()
clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
clf_DT = DecisionTreeClassifier()
clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

clf_SVM.fit(X_train_new, y_train_new)
clf_KNN.fit(X_train_new, y_train_new)
clf_LR.fit(X_train_new, y_train_new)
clf_DT.fit(X_train_new, y_train_new)
clf_RF.fit(X_train_new, y_train_new)
clf_LightGBM.fit(X_train_new, y_train_new)
clf_Adaboost.fit(X_train_new, y_train_new)
clf_GBDT.fit(X_train_new, y_train_new)

y_pred_SVM = clf_SVM.predict(X_test)
y_pred_KNN = clf_KNN.predict(X_test)
y_pred_LR = clf_LR.predict(X_test)
y_pred_DT = clf_DT.predict(X_test)
y_pred_RF = clf_RF.predict(X_test)
y_pred_LightGBM = clf_LightGBM.predict(X_test)
y_pred_Adaboost = clf_Adaboost.predict(X_test)
y_pred_GBDT = clf_GBDT.predict(X_test)

y train ratio: 1:1


In [16]:
with open(f"{os.path.dirname(os.getcwd())}/result/{write_file}", "a", newline="") as f:
    csv_writer = csv.writer(f)
    
    csv_writer.writerow(["SMOTE", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
    csv_writer.writerow(["SMOTE", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
    csv_writer.writerow(["SMOTE", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
    csv_writer.writerow(["SMOTE", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
    csv_writer.writerow(["SMOTE", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
    csv_writer.writerow(["SMOTE", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
    csv_writer.writerow(["SMOTE", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
    csv_writer.writerow(["SMOTE", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

### SVMSMOTE Oversampling

In [17]:
# SVMSMOTE oversampling run - SVMSMOTE oversampling technique
# inputs: X_train_random, y_train_random, X_test, y_test

rt, X_train_new, y_train_new = SVMSMOTEOversampling(X_train=X_train, y_train=y_train)

# scale data
scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

# create models
clf_SVM = SVC()
clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
clf_DT = DecisionTreeClassifier()
clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

clf_SVM.fit(X_train_new, y_train_new)
clf_KNN.fit(X_train_new, y_train_new)
clf_LR.fit(X_train_new, y_train_new)
clf_DT.fit(X_train_new, y_train_new)
clf_RF.fit(X_train_new, y_train_new)
clf_LightGBM.fit(X_train_new, y_train_new)
clf_Adaboost.fit(X_train_new, y_train_new)
clf_GBDT.fit(X_train_new, y_train_new)

y_pred_SVM = clf_SVM.predict(X_test)
y_pred_KNN = clf_KNN.predict(X_test)
y_pred_LR = clf_LR.predict(X_test)
y_pred_DT = clf_DT.predict(X_test)
y_pred_RF = clf_RF.predict(X_test)
y_pred_LightGBM = clf_LightGBM.predict(X_test)
y_pred_Adaboost = clf_Adaboost.predict(X_test)
y_pred_GBDT = clf_GBDT.predict(X_test)

y train ratio: 1:1


In [18]:
with open(f"{os.path.dirname(os.getcwd())}/result/{write_file}", "a", newline="") as f:
    csv_writer = csv.writer(f)
    
    csv_writer.writerow(["SVMSMOTE", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
    csv_writer.writerow(["SVMSMOTE", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
    csv_writer.writerow(["SVMSMOTE", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
    csv_writer.writerow(["SVMSMOTE", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
    csv_writer.writerow(["SVMSMOTE", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
    csv_writer.writerow(["SVMSMOTE", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
    csv_writer.writerow(["SVMSMOTE", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
    csv_writer.writerow(["SVMSMOTE", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

### SMOTUNED Oversampling

In [19]:
# SMOTUNED oversampling run - SMOTUNED oversampling technique
# inputs: X_train_random, y_train_random, X_test, y_test

rt_SVM, X_train_new_SVM, y_train_new_SVM = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                y_train=y_train, y_test=y_test, model="SVM")

scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new_SVM), columns=X_train_new_SVM.columns)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

print("y train ratio of SVM: 1:" + str(round(y_train_new_SVM.value_counts()[0] / y_train_new_SVM.value_counts()[1])))

rt_KNN, X_train_new_KNN, y_train_new_KNN = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                y_train=y_train, y_test=y_test, model="KNN")

scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new_KNN), columns=X_train_new_KNN.columns)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

print("y train ratio of KNN: 1:" + str(round(y_train_new_KNN.value_counts()[0] / y_train_new_KNN.value_counts()[1])))

rt_LR, X_train_new_LR, y_train_new_LR = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                             y_train=y_train, y_test=y_test, model="LR")

scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new_LR), columns=X_train_new_LR.columns)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

print("y train ratio of LR: 1:" + str(round(y_train_new_LR.value_counts()[0] / y_train_new_LR.value_counts()[1])))

rt_DT, X_train_new_DT, y_train_new_DT = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                             y_train=y_train, y_test=y_test, model="DT")

scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new_DT), columns=X_train_new_DT.columns)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

print("y train ratio of DT: 1:" + str(round(y_train_new_DT.value_counts()[0] / y_train_new_DT.value_counts()[1])))

rt_RF, X_train_new_RF, y_train_new_RF = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                             y_train=y_train, y_test=y_test, model="RF")

scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new_RF), columns=X_train_new_RF.columns)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

print("y train ratio of RF: 1:" + str(round(y_train_new_RF.value_counts()[0] / y_train_new_RF.value_counts()[1])))

rt_LightGBM, X_train_new_LightGBM, y_train_new_LightGBM = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                               y_train=y_train, y_test=y_test, model="LightGBM")

scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new_LightGBM), columns=X_train_new_LightGBM.columns)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

print("y train ratio of LightGBM: 1:" + str(round(y_train_new_LightGBM.value_counts()[0] / y_train_new_LightGBM.value_counts()[1])))

rt_Adaboost, X_train_new_Adaboost, y_train_new_Adaboost = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                               y_train=y_train, y_test=y_test, model="Adaboost")

scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new_Adaboost), columns=X_train_new_Adaboost.columns)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

print("y train ratio of Adaboost: 1:" + str(round(y_train_new_Adaboost.value_counts()[0] / y_train_new_Adaboost.value_counts()[1])))

rt_GBDT, X_train_new_GBDT, y_train_new_GBDT = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                   y_train=y_train, y_test=y_test, model="GBDT")

scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new_GBDT), columns=X_train_new_GBDT.columns)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

print("y train ratio of GBDT: 1:" + str(round(y_train_new_GBDT.value_counts()[0] / y_train_new_GBDT.value_counts()[1])))

# create models
clf_SVM = SVC()
clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
clf_DT = DecisionTreeClassifier()
clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

clf_SVM.fit(X_train_new_SVM, y_train_new_SVM)
clf_KNN.fit(X_train_new_KNN, y_train_new_KNN)
clf_LR.fit(X_train_new_LR, y_train_new_LR)
clf_DT.fit(X_train_new_DT, y_train_new_DT)
clf_RF.fit(X_train_new_RF, y_train_new_RF)
clf_LightGBM.fit(X_train_new_LightGBM, y_train_new_LightGBM)
clf_Adaboost.fit(X_train_new_Adaboost, y_train_new_Adaboost)
clf_GBDT.fit(X_train_new_GBDT, y_train_new_GBDT)

y_pred_SVM = clf_SVM.predict(X_test)
y_pred_KNN = clf_KNN.predict(X_test)
y_pred_LR = clf_LR.predict(X_test)
y_pred_DT = clf_DT.predict(X_test)
y_pred_RF = clf_RF.predict(X_test)
y_pred_LightGBM = clf_LightGBM.predict(X_test)
y_pred_Adaboost = clf_Adaboost.predict(X_test)
y_pred_GBDT = clf_GBDT.predict(X_test)

y train ratio of SVM: 1:1
y train ratio of KNN: 1:1
y train ratio of LR: 1:1
y train ratio of DT: 1:1
y train ratio of RF: 1:1
y train ratio of LightGBM: 1:1
y train ratio of Adaboost: 1:1
y train ratio of GBDT: 1:1


In [20]:
with open(f"{os.path.dirname(os.getcwd())}/result/{write_file}", "a", newline="") as f:
    csv_writer = csv.writer(f)
    
    csv_writer.writerow(["SMOTUNED", rt_SVM, "SVM"] + evaluate_result(y_pred_SVM, y_test))
    csv_writer.writerow(["SMOTUNED", rt_KNN, "KNN"] + evaluate_result(y_pred_KNN, y_test))
    csv_writer.writerow(["SMOTUNED", rt_LR, "LR"] + evaluate_result(y_pred_LR, y_test))
    csv_writer.writerow(["SMOTUNED", rt_DT, "DT"] + evaluate_result(y_pred_DT, y_test))
    csv_writer.writerow(["SMOTUNED", rt_RF, "RF"] + evaluate_result(y_pred_RF, y_test))
    csv_writer.writerow(["SMOTUNED", rt_LightGBM, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
    csv_writer.writerow(["SMOTUNED", rt_Adaboost, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
    csv_writer.writerow(["SMOTUNED", rt_GBDT, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

### DAZZLE Oversampling

In [21]:
# DAZZLE oversampling run - DAZZLE oversampling technique
# inputs: X_train_random, y_train_random, X_test, y_test

scaler = StandardScaler()
X_train_GAN = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
rt, X_train_new, y_train_new = DAZZLEOversampling(X_train=X_train_GAN, y_train=y_train)

# scale data
scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

# create models
clf_SVM = SVC()
clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
clf_DT = DecisionTreeClassifier()
clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

clf_SVM.fit(X_train_new, y_train_new)
clf_KNN.fit(X_train_new, y_train_new)
clf_LR.fit(X_train_new, y_train_new)
clf_DT.fit(X_train_new, y_train_new)
clf_RF.fit(X_train_new, y_train_new)
clf_LightGBM.fit(X_train_new, y_train_new)
clf_Adaboost.fit(X_train_new, y_train_new)
clf_GBDT.fit(X_train_new, y_train_new)

y_pred_SVM = clf_SVM.predict(X_test)
y_pred_KNN = clf_KNN.predict(X_test)
y_pred_LR = clf_LR.predict(X_test)
y_pred_DT = clf_DT.predict(X_test)
y_pred_RF = clf_RF.predict(X_test)
y_pred_LightGBM = clf_LightGBM.predict(X_test)
y_pred_Adaboost = clf_Adaboost.predict(X_test)
y_pred_GBDT = clf_GBDT.predict(X_test)

  0%|                                                                           | 0/100 [00:00<?, ?trial/s, best loss=?]

2023-09-19 12:50:31.798203: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-19 12:50:31.798478: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


100%|██████████████████████████████████████████████| 100/100 [00:59<00:00,  1.68trial/s, best loss: -0.8443986178332092]
Best Hyperparameters: {'batch_size': 16, 'discriminator_activation_fn': <function leaky_relu at 0x7f33e8934160>, 'discriminator_layer_normalization': True, 'discriminator_lr': 0.0009917984440465412, 'discriminator_optimizer': <class 'keras.src.optimizers.sgd.SGD'>, 'epochs': 15, 'generator_activation_fn': <function leaky_relu at 0x7f33e8934160>, 'generator_layer_normalization': False, 'generator_lr': 0.0035633076617442015, 'generator_optimizer': <class 'keras.src.optimizers.adamax.Adamax'>}
Best G-Measure: 0.8443986178332092
25/25 [==============================] - 0s 2ms/step


/home/lingxiao/diveplane/lib/python3.8/site-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


y train ratio: 1:1


In [22]:
with open(f"{os.path.dirname(os.getcwd())}/result/{write_file}", "a", newline="") as f:
    csv_writer = csv.writer(f)
    
    csv_writer.writerow(["DAZZLE", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
    csv_writer.writerow(["DAZZLE", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
    csv_writer.writerow(["DAZZLE", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
    csv_writer.writerow(["DAZZLE", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
    csv_writer.writerow(["DAZZLE", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
    csv_writer.writerow(["DAZZLE", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
    csv_writer.writerow(["DAZZLE", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
    csv_writer.writerow(["DAZZLE", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

### WGAN Oversampling

In [23]:
# WGAN oversampling run - WGAN oversampling technique
# inputs: X_train_random, y_train_random, X_test, y_test

scaler = StandardScaler()
X_train_GAN = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
rt, X_train_new, y_train_new = GANOversampling(X_train=X_train_GAN, y_train=y_train)

# scale data
scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

# create models
clf_SVM = SVC()
clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
clf_DT = DecisionTreeClassifier()
clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

clf_SVM.fit(X_train_new, y_train_new)
clf_KNN.fit(X_train_new, y_train_new)
clf_LR.fit(X_train_new, y_train_new)
clf_DT.fit(X_train_new, y_train_new)
clf_RF.fit(X_train_new, y_train_new)
clf_LightGBM.fit(X_train_new, y_train_new)
clf_Adaboost.fit(X_train_new, y_train_new)
clf_GBDT.fit(X_train_new, y_train_new)

y_pred_SVM = clf_SVM.predict(X_test)
y_pred_KNN = clf_KNN.predict(X_test)
y_pred_LR = clf_LR.predict(X_test)
y_pred_DT = clf_DT.predict(X_test)
y_pred_RF = clf_RF.predict(X_test)
y_pred_LightGBM = clf_LightGBM.predict(X_test)
y_pred_Adaboost = clf_Adaboost.predict(X_test)
y_pred_GBDT = clf_GBDT.predict(X_test)

MAX CLASS 632
CLASS ID 1
Epoch 1/150 completed. Gen loss: -0.01348569430410862. Desc loss_real: 0.019305530935525894. Desc loss_fake: 0.01348569430410862
Epoch 51/150 completed. Gen loss: -0.0016907525714486837. Desc loss_real: 0.011242654174566269. Desc loss_fake: 0.0016907525714486837
Epoch 101/150 completed. Gen loss: -0.0033969709184020758. Desc loss_real: 0.014724577777087688. Desc loss_fake: 0.0033969709184020758


/mnt/e/Research/SyntheticData/src/data_imbalance_src/Imbalance_Farou2022/data_generation.py:119: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(synthetic_data)
/mnt/e/Research/SyntheticData/src/data_imbalance_src/Imbalance_Farou2022/data_generation.py:133: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_sample = X_sample.append(X_train)
/mnt/e/Research/SyntheticData/src/data_imbalance_src/Imbalance_Farou2022/data_generation.py:135: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_sample = X_sample.drop(tar, 1)


y train ratio: 1:1


In [24]:
with open(f"{os.path.dirname(os.getcwd())}/result/{write_file}", "a", newline="") as f:
    csv_writer = csv.writer(f)
    
    csv_writer.writerow(["WGAN", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
    csv_writer.writerow(["WGAN", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
    csv_writer.writerow(["WGAN", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
    csv_writer.writerow(["WGAN", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
    csv_writer.writerow(["WGAN", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
    csv_writer.writerow(["WGAN", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
    csv_writer.writerow(["WGAN", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
    csv_writer.writerow(["WGAN", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

### Random Projection Oversampling

In [25]:
# Random projection oversampling run - Random projection oversampling technique
# inputs: X_train_random, y_train_random, X_test, y_test

rt, X_train_new, y_train_new = RandomProjectionOversampling(X_train=X_train, y_train=y_train)

# scale data
scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

# create models
clf_SVM = SVC()
clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
clf_DT = DecisionTreeClassifier()
clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

clf_SVM.fit(X_train_new, y_train_new)
clf_KNN.fit(X_train_new, y_train_new)
clf_LR.fit(X_train_new, y_train_new)
clf_DT.fit(X_train_new, y_train_new)
clf_RF.fit(X_train_new, y_train_new)
clf_LightGBM.fit(X_train_new, y_train_new)
clf_Adaboost.fit(X_train_new, y_train_new)
clf_GBDT.fit(X_train_new, y_train_new)

y_pred_SVM = clf_SVM.predict(X_test)
y_pred_KNN = clf_KNN.predict(X_test)
y_pred_LR = clf_LR.predict(X_test)
y_pred_DT = clf_DT.predict(X_test)
y_pred_RF = clf_RF.predict(X_test)
y_pred_LightGBM = clf_LightGBM.predict(X_test)
y_pred_Adaboost = clf_Adaboost.predict(X_test)
y_pred_GBDT = clf_GBDT.predict(X_test)

y train ratio: 1:1


In [26]:
with open(f"{os.path.dirname(os.getcwd())}/result/{write_file}", "a", newline="") as f:
    csv_writer = csv.writer(f)
    
    csv_writer.writerow(["RP", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
    csv_writer.writerow(["RP", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
    csv_writer.writerow(["RP", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
    csv_writer.writerow(["RP", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
    csv_writer.writerow(["RP", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
    csv_writer.writerow(["RP", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
    csv_writer.writerow(["RP", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
    csv_writer.writerow(["RP", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

### Diveplane Oversampling

In [61]:
from diveplane.utilities import infer_feature_attributes
from diveplane.geminai import Geminai

In [62]:
tar = y_train.name
conditions = [{tar: 1},
              {tar: 0}] * (int(X_train.shape[0] / 2))

X_train[tar] = y_train
# partial_features = {"CLLC": {'type': "continuous"}}
features = infer_feature_attributes(X_train)
for f_name, f_value in features.items():
    if f_value["type"] == "nominal":
        f_value["non_sensitive"] = True

start_time = time.time()
g = Geminai()
g.train(X_train, features=features)

gen_df = g.synthesize_cases(
    n_samples=len(conditions),
    case_context_values_maps=conditions,
    generate_new_cases="no"
)

rt = time.time() - start_time

X_train = X_train.iloc[:, :-1]
X_train_new = gen_df.iloc[:, :-1]
y_train_new = gen_df.iloc[:, -1]

/home/lingxiao/diveplane/lib/python3.8/site-packages/diveplane/geminai/base.py:376: UserWarning: The value of the parameter `generate_new_cases` is set as 'no' and not 'always', which means it is possible that some cases that are very similar to the original data may be returned. To prevent this from happening, please set generate_new_cases to 'always'.
  warnings.warn(


In [63]:
# scale data
scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

# create models
clf_SVM = SVC()
clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
clf_DT = DecisionTreeClassifier()
clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

clf_SVM.fit(X_train_new, y_train_new)
clf_KNN.fit(X_train_new, y_train_new)
clf_LR.fit(X_train_new, y_train_new)
clf_DT.fit(X_train_new, y_train_new)
clf_RF.fit(X_train_new, y_train_new)
clf_LightGBM.fit(X_train_new, y_train_new)
clf_Adaboost.fit(X_train_new, y_train_new)
clf_GBDT.fit(X_train_new, y_train_new)

y_pred_SVM = clf_SVM.predict(X_test)
y_pred_KNN = clf_KNN.predict(X_test)
y_pred_LR = clf_LR.predict(X_test)
y_pred_DT = clf_DT.predict(X_test)
y_pred_RF = clf_RF.predict(X_test)
y_pred_LightGBM = clf_LightGBM.predict(X_test)
y_pred_Adaboost = clf_Adaboost.predict(X_test)
y_pred_GBDT = clf_GBDT.predict(X_test)

y train ratio: 1:1


In [64]:
with open(f"{os.path.dirname(os.getcwd())}/result/{write_file}", "a", newline="") as f:
    csv_writer = csv.writer(f)
    
    csv_writer.writerow(["Diveplane", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
    csv_writer.writerow(["Diveplane", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
    csv_writer.writerow(["Diveplane", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
    csv_writer.writerow(["Diveplane", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
    csv_writer.writerow(["Diveplane", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
    csv_writer.writerow(["Diveplane", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
    csv_writer.writerow(["Diveplane", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
    csv_writer.writerow(["Diveplane", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

NameError: name 'write_file' is not defined

In [65]:
tar = y_train.name
n_cases_diff = y_train.value_counts()[0] - y_train.value_counts()[1]
conditions = [{tar: 1}] * n_cases_diff

X_train_diveplane = X_train.copy()
y_train_diveplane = y_train.copy()
X_train_diveplane[tar] = y_train_diveplane

features = infer_feature_attributes(X_train_diveplane)
for f_name, f_value in features.items():
    if f_value["type"] == "nominal":
        f_value["non_sensitive"] = True

start_time = time.time()
g = Geminai()
g.train(X_train_diveplane, features=features)

gen_df = g.synthesize_cases(
    n_samples=len(conditions),
    case_context_values_maps=conditions,
    desired_conviction=5,
    generate_new_cases="no"
)

rt = time.time() - start_time

X_train_new = pd.concat([X_train_diveplane, gen_df], ignore_index=True, axis=0)
y_train_new = X_train_new.iloc[:, -1]
X_train_new = X_train_new.iloc[:, :-1]

/home/lingxiao/diveplane/lib/python3.8/site-packages/diveplane/geminai/base.py:376: UserWarning: The value of the parameter `generate_new_cases` is set as 'no' and not 'always', which means it is possible that some cases that are very similar to the original data may be returned. To prevent this from happening, please set generate_new_cases to 'always'.
  warnings.warn(


In [67]:
y_train_new.value_counts()[0]

4293

In [68]:
y_train_new.value_counts()[1]

4293

In [66]:
y_train_new.isnull().values.any()

False

### DS Oversampling

In [31]:
from DataSynthesizer.DataDescriber import DataDescriber
from DataSynthesizer.DataGenerator import DataGenerator
from DataSynthesizer.ModelInspector import ModelInspector
from DataSynthesizer.lib.utils import read_json_file, display_bayesian_network
mode = "independent_attribute_mode"

col = X_train.columns
tar = y_train.name
X_train[tar] = y_train
write_df = X_train[X_train[tar] == 1]
write_df = write_df.iloc[:, :-1]
write_df.to_csv(f"{os.path.dirname(os.getcwd())}/extra/js_vuln_pos_df.csv", index=False)
X_train = X_train.iloc[:, :-1]

threshold = 20
num_tuples_to_generate = int(y_train.value_counts()[0] - y_train.value_counts()[1])

start_time = time.time()

description_file = f"{os.path.dirname(os.getcwd())}/extra/js_vuln.json"
describer = DataDescriber(category_threshold=threshold)
describer.describe_dataset_in_independent_attribute_mode(
    dataset_file=f"{os.path.dirname(os.getcwd())}/extra/js_vuln_pos_df.csv"
)
describer.save_dataset_description_to_file(description_file)

generator = DataGenerator()
generator.generate_dataset_in_independent_mode(num_tuples_to_generate, description_file)
generator.save_synthetic_data(f"{os.path.dirname(os.getcwd())}/extra/js_vuln_syn_df.csv")

rt = time.time() - start_time

X_train_new = pd.read_csv(f"{os.path.dirname(os.getcwd())}/extra/js_vuln_syn_df.csv").to_numpy()
y_train_new = np.ones(num_tuples_to_generate)
X_train_new = pd.DataFrame(np.vstack((X_train.to_numpy(), X_train_new)), columns=col)
y_train_new = pd.Series(np.hstack((y_train.to_numpy(), y_train_new)), name=tar)

In [32]:
# scale data
scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

# create models
clf_SVM = SVC()
clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
clf_DT = DecisionTreeClassifier()
clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

clf_SVM.fit(X_train_new, y_train_new)
clf_KNN.fit(X_train_new, y_train_new)
clf_LR.fit(X_train_new, y_train_new)
clf_DT.fit(X_train_new, y_train_new)
clf_RF.fit(X_train_new, y_train_new)
clf_LightGBM.fit(X_train_new, y_train_new)
clf_Adaboost.fit(X_train_new, y_train_new)
clf_GBDT.fit(X_train_new, y_train_new)

y_pred_SVM = clf_SVM.predict(X_test)
y_pred_KNN = clf_KNN.predict(X_test)
y_pred_LR = clf_LR.predict(X_test)
y_pred_DT = clf_DT.predict(X_test)
y_pred_RF = clf_RF.predict(X_test)
y_pred_LightGBM = clf_LightGBM.predict(X_test)
y_pred_Adaboost = clf_Adaboost.predict(X_test)
y_pred_GBDT = clf_GBDT.predict(X_test)

y train ratio: 1:1


In [33]:
with open(f"{os.path.dirname(os.getcwd())}/result/{write_file}", "a", newline="") as f:
    csv_writer = csv.writer(f)
    
    csv_writer.writerow(["DS", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
    csv_writer.writerow(["DS", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
    csv_writer.writerow(["DS", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
    csv_writer.writerow(["DS", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
    csv_writer.writerow(["DS", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
    csv_writer.writerow(["DS", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
    csv_writer.writerow(["DS", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
    csv_writer.writerow(["DS", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

### SDV Oversampling

In [34]:
from sdv.metadata import SingleTableMetadata
from sdv.lite import SingleTablePreset
from sdv.single_table import CTGANSynthesizer
from sdv.single_table import GaussianCopulaSynthesizer

In [35]:
col = X_train.columns
tar = y_train.name
num_tuples_to_generate = int(y_train.value_counts()[0] - y_train.value_counts()[1])
X_train[tar] = y_train
pos_df = X_train[X_train[tar] == 1]
pos_df = pos_df.iloc[:, :-1]
X_train = X_train.iloc[:, :-1]

metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=pos_df)

In [36]:
start_time = time.time()
syn1 = SingleTablePreset(metadata, name="FAST_ML")
syn1.fit(data=pos_df)
X_train_new = syn1.sample(num_rows=num_tuples_to_generate).to_numpy()

rt = time.time() - start_time

X_train_new = pd.DataFrame(np.vstack((X_train.to_numpy(), X_train_new)), columns=col)
y_train_new = np.ones(num_tuples_to_generate)
y_train_new = pd.Series(np.hstack((y_train.to_numpy(), y_train_new)), name=tar)

# scale data
scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

# create models
clf_SVM = SVC()
clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
clf_DT = DecisionTreeClassifier()
clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

clf_SVM.fit(X_train_new, y_train_new)
clf_KNN.fit(X_train_new, y_train_new)
clf_LR.fit(X_train_new, y_train_new)
clf_DT.fit(X_train_new, y_train_new)
clf_RF.fit(X_train_new, y_train_new)
clf_LightGBM.fit(X_train_new, y_train_new)
clf_Adaboost.fit(X_train_new, y_train_new)
clf_GBDT.fit(X_train_new, y_train_new)

y_pred_SVM = clf_SVM.predict(X_test)
y_pred_KNN = clf_KNN.predict(X_test)
y_pred_LR = clf_LR.predict(X_test)
y_pred_DT = clf_DT.predict(X_test)
y_pred_RF = clf_RF.predict(X_test)
y_pred_LightGBM = clf_LightGBM.predict(X_test)
y_pred_Adaboost = clf_Adaboost.predict(X_test)
y_pred_GBDT = clf_GBDT.predict(X_test)

y train ratio: 1:1


In [37]:
with open(f"{os.path.dirname(os.getcwd())}/result/{write_file}", "a", newline="") as f:
    csv_writer = csv.writer(f)
    
    csv_writer.writerow(["SDV_FASTML", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
    csv_writer.writerow(["SDV_FASTML", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
    csv_writer.writerow(["SDV_FASTML", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
    csv_writer.writerow(["SDV_FASTML", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
    csv_writer.writerow(["SDV_FASTML", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
    csv_writer.writerow(["SDV_FASTML", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
    csv_writer.writerow(["SDV_FASTML", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
    csv_writer.writerow(["SDV_FASTML", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

In [38]:
start_time = time.time()
syn2 = GaussianCopulaSynthesizer(metadata)
syn2.fit(data=pos_df)
X_train_new = syn2.sample(num_rows=num_tuples_to_generate).to_numpy()

rt = time.time() - start_time

X_train_new = pd.DataFrame(np.vstack((X_train.to_numpy(), X_train_new)), columns=col)
y_train_new = np.ones(num_tuples_to_generate)
y_train_new = pd.Series(np.hstack((y_train.to_numpy(), y_train_new)), name=tar)

# scale data
scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

# create models
clf_SVM = SVC()
clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
clf_DT = DecisionTreeClassifier()
clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

clf_SVM.fit(X_train_new, y_train_new)
clf_KNN.fit(X_train_new, y_train_new)
clf_LR.fit(X_train_new, y_train_new)
clf_DT.fit(X_train_new, y_train_new)
clf_RF.fit(X_train_new, y_train_new)
clf_LightGBM.fit(X_train_new, y_train_new)
clf_Adaboost.fit(X_train_new, y_train_new)
clf_GBDT.fit(X_train_new, y_train_new)

y_pred_SVM = clf_SVM.predict(X_test)
y_pred_KNN = clf_KNN.predict(X_test)
y_pred_LR = clf_LR.predict(X_test)
y_pred_DT = clf_DT.predict(X_test)
y_pred_RF = clf_RF.predict(X_test)
y_pred_LightGBM = clf_LightGBM.predict(X_test)
y_pred_Adaboost = clf_Adaboost.predict(X_test)
y_pred_GBDT = clf_GBDT.predict(X_test)

y train ratio: 1:1


In [39]:
with open(f"{os.path.dirname(os.getcwd())}/result/{write_file}", "a", newline="") as f:
    csv_writer = csv.writer(f)
    
    csv_writer.writerow(["SDV_GC", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
    csv_writer.writerow(["SDV_GC", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
    csv_writer.writerow(["SDV_GC", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
    csv_writer.writerow(["SDV_GC", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
    csv_writer.writerow(["SDV_GC", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
    csv_writer.writerow(["SDV_GC", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
    csv_writer.writerow(["SDV_GC", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
    csv_writer.writerow(["SDV_GC", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

In [40]:
start_time = time.time()
syn3 = CTGANSynthesizer(metadata)
syn3.fit(data=pos_df)
X_train_new = syn3.sample(num_rows=num_tuples_to_generate).to_numpy()

rt = time.time() - start_time

X_train_new = pd.DataFrame(np.vstack((X_train.to_numpy(), X_train_new)), columns=col)
y_train_new = np.ones(num_tuples_to_generate)
y_train_new = pd.Series(np.hstack((y_train.to_numpy(), y_train_new)), name=tar)

# scale data
scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

# create models
clf_SVM = SVC()
clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
clf_DT = DecisionTreeClassifier()
clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

clf_SVM.fit(X_train_new, y_train_new)
clf_KNN.fit(X_train_new, y_train_new)
clf_LR.fit(X_train_new, y_train_new)
clf_DT.fit(X_train_new, y_train_new)
clf_RF.fit(X_train_new, y_train_new)
clf_LightGBM.fit(X_train_new, y_train_new)
clf_Adaboost.fit(X_train_new, y_train_new)
clf_GBDT.fit(X_train_new, y_train_new)

y_pred_SVM = clf_SVM.predict(X_test)
y_pred_KNN = clf_KNN.predict(X_test)
y_pred_LR = clf_LR.predict(X_test)
y_pred_DT = clf_DT.predict(X_test)
y_pred_RF = clf_RF.predict(X_test)
y_pred_LightGBM = clf_LightGBM.predict(X_test)
y_pred_Adaboost = clf_Adaboost.predict(X_test)
y_pred_GBDT = clf_GBDT.predict(X_test)

/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missi

/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missi

/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missi

y train ratio: 1:1


In [41]:
with open(f"{os.path.dirname(os.getcwd())}/result/{write_file}", "a", newline="") as f:
    csv_writer = csv.writer(f)
    
    csv_writer.writerow(["SDV_GAN", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
    csv_writer.writerow(["SDV_GAN", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
    csv_writer.writerow(["SDV_GAN", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
    csv_writer.writerow(["SDV_GAN", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
    csv_writer.writerow(["SDV_GAN", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
    csv_writer.writerow(["SDV_GAN", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
    csv_writer.writerow(["SDV_GAN", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
    csv_writer.writerow(["SDV_GAN", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))